In [1]:
# scikit learn install
!pip install scikit-learn

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np

In [3]:
iris = load_iris()
x = iris.data
y = iris.target

In [4]:
# 붓꽃의 꽃받침과 꽃잎의 [길이, 너비]에 대한 값 -> 붓꽃의 종류 setosa / versicolor / virginica
print(x.shape, y.shape)

(150, 4) (150,)


In [5]:
def norm_maxmin(data, out_min=0, out_max=255):
    
    gradient = (out_max - out_min) / (np.max(data) - np.min(data))
    bias = out_min - np.min(data) * gradient
    out = data * gradient + bias

    return out

In [6]:
for it_col in range(x.shape[1]):
    x[:, it_col] = norm_maxmin(x[:, it_col], 0, 255)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [8]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(135, 4) (15, 4) (135,) (15,)


In [9]:
x_train = x_train.reshape(-1, 15, 4)
y_train = y_train.reshape(-1, 15)

In [10]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(9, 15, 4) (15, 4) (9, 15) (15,)


In [11]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

In [12]:
class FCN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FCN, self).__init__()
        self.fc = nn.Linear(input_dim, 12)
        self.fc2 = nn.Linear(12, num_classes)

    def forward(self, x):
        x = torch.sigmoid(self.fc(x))
        return torch.sigmoid(self.fc2(x))

In [13]:
model = FCN(input_dim=x_train.shape[2], num_classes=3)
print(model)

FCN(
  (fc): Linear(in_features=4, out_features=12, bias=True)
  (fc2): Linear(in_features=12, out_features=3, bias=True)
)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
epochs = 10000
for e in range(epochs):
    for b in range(len(x_train)):
        outputs = model(x_train[b])
        loss = criterion(outputs, y_train[b])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (e+1) % 1000 == 0:
        print(f'Epoch {e + 1} / {epochs}, Loss : {loss.item()}')

Epoch 1000 / 10000, Loss : 0.6564655900001526
Epoch 2000 / 10000, Loss : 0.5964353680610657
Epoch 3000 / 10000, Loss : 0.590671181678772
Epoch 4000 / 10000, Loss : 0.5808330774307251
Epoch 5000 / 10000, Loss : 0.5715911388397217
Epoch 6000 / 10000, Loss : 0.5659119486808777
Epoch 7000 / 10000, Loss : 0.5596054792404175
Epoch 8000 / 10000, Loss : 0.5585200786590576
Epoch 9000 / 10000, Loss : 0.5575879812240601
Epoch 10000 / 10000, Loss : 0.5569316148757935


In [16]:
# 테스트
with torch.no_grad():

    model.eval()

    correct = 0
    total = 0
    outputs = model(x_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()
    print('Accuracy of the model on test data: {} %'.format(100 * correct / total))


Accuracy of the model on test data: 93.33333333333333 %
